In [ ]:
from functions import sample_uniform_parameters, sample_correlated_parameters, output_from_parameters, output_from_parameters_with_noise, obtain_theta, perform_CNLS_LASSO, obtain_beta_unique, add_random_variables
from pystoned.plot import plot2d
import numpy as np
import pandas as pd

### Sample the true parameters X (iid vs. Correlated), and create Y
### Obtain theta from the results

In [ ]:
x = sample_uniform_parameters(2,25, min_value=1, max_value=50, seed=2)
y_log_true = output_from_parameters(x, cons = 3)
y_log = output_from_parameters_with_noise(x, cons=3, var=0.1)
x = add_random_variables(x, 7, min_value = 30, max_value = 40, seed=3)

In [ ]:
theta = obtain_theta(x, y_true)

### Perform the SCNLS-LASSO regression

In [ ]:
model_cnls = perform_CNLS_LASSO(x=x, y=y, eta=0)
beta = model_cnls.get_beta()
alpha = model_cnls.get_alpha()
beta = pd.DataFrame(beta).round(3)
beta.loc['Total',:] = beta.sum(axis=0)
display(beta)
# beta_unique = obtain_beta_unique(x, alpha, beta).T.round(2).rename(columns={0:'Alpha'})
# beta_unique.loc['Total',:] = beta_unique.sum(axis=0)
# display(beta_unique)
# plot2d(model_cnls, x_select=0, label_name="CNLS", fig_name='CNLS_frontier')

In [ ]:
plot2d(model_cnls, x_select=0, label_name="CNLS", fig_name='CNLS_frontier')